In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch import nn
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore") 
import sys
sys.path.append('../scripts/')
from metrics import find_metrics, find_metrics_macro
from eval_model import EvalModel

/home/paperspace/Documents/tensor_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
csv_file = pd.read_csv('../../ml_research_data/dolphin_dataset/files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../ml_research_data/dolphin_dataset/files/train_images/' + x)

In [3]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size, random_state=42)
    train, val = train_test_split(train, test_size=test_size, random_state=42)
    return train, val, test

In [4]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [5]:
encoder = encoding_data(csv_file)

In [6]:
train, val, test = split_datasets(csv_file, test_size = 0.01)

In [7]:
obj = EvalModel(val, '../../ml_research_data/dolphin_weights/torch_with_momentum/49.pth', encoder, (1024, 1024))

Using cache found in /home/paperspace/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
obj.eval_mdoel()

100%|██████████| 253/253 [00:33<00:00,  7.49it/s]

222.29233155900147
Loss: 0.8786258164387409
Prec: 0.04677207095055241
Rec: 0.046508566325478874
